In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, Dense
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from google.colab import files
#uploaded = files.upload()
import pandas as pd
import kagglehub
from deep_translator import GoogleTranslator
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer


In [ ]:
import pandas as pd
#df_vader = pd.read_parquet(r'C:\Users\ovelasqueza\Downloads\IA.Parquet', engine='pyarrow')
from google.colab import files

# Subir el archivo .parquet desde tu computadora
uploaded = files.upload()

# Asumiendo que has subido un archivo llamado "archivo.parquet"
df_vader = pd.read_parquet('IA.parquet', engine='pyarrow')

# Mostrar las primeras filas del DataFrame
print(df_vader.head())



Saving IA.parquet to IA (4).parquet
  store                assigned_at  \
0  None 2023-02-21 07:18:07.598950   
1  FN60 2023-02-20 12:25:30.652022   
2  T002 2023-02-18 14:10:31.610430   
3  T013 2023-02-23 12:40:51.640983   
4  FN54 2023-02-20 17:10:31.230531   

                                              answer  
0                                         comentario  
1  Todo lo hacen perfecto,, gracias por su atención   
2  Offcorss...siempre me ha parecido una buena al...  
3  Me encanta las tiendas el servicio y sus produ...  
4                                Ya no ay probadores  


In [ ]:
!pip install deep-translator
!pip install vaderSentiment
from deep_translator import GoogleTranslator
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

# Inicializar el traductor y el analizador de sentimientos
translator = GoogleTranslator(source='spanish', target='english')
sia = SentimentIntensityAnalyzer()



In [ ]:
print(df_vader.isnull().values.any())
df_vader['answer'].fillna('', inplace=True)

True


<ipython-input-23-f7b8d0489e75>:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_vader['answer'].fillna('', inplace=True)


In [ ]:
# Translate each answer individually and store the results in a list

df_vader_In = []
for answer in df_vader['answer']:
    # Check if the answer exceeds the character limit and split if needed
    if len(answer) > 5000:
        # Split the answer into chunks of 5000 characters or less
        chunks = [answer[i:i + 5000] for i in range(0, len(answer), 5000)]
        translated_chunks = [translator.translate(chunk) for chunk in chunks]
        # Join the translated chunks back together
        translated_answer = ' '.join(translated_chunks)
    else:
        translated_answer = translator.translate(answer)
    df_vader_In.append(translated_answer)

# Convert the translated answers list into a pandas Series
#df_vader_In = pd.Series(df_vader_In)

In [ ]:
#grabo el dataset o el dico duro o en drive para no tener que volver a traducirlo
num_filas = len(df_vader_In)
print(f"El número de filas es: {num_filas}")
import os
try:
#    este lo graba temporalmente en google colbas
    df_vader_In.to_csv(r'dataset.csv', index=False)
#  este lo graba en el archivo descargas del pc luego se puede volver a cargar
#    files.download(r'C:\temp\dataset.csv')
    print("File saved successfully!")
except Exception as e:
    print(f"An error occurred while creating the file: {e}")




El número de filas es: 17050


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

File saved successfully!


In [ ]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

# Lista de stopwords en inglés
stop_words = set(stopwords.words('english'))

# Eliminar stopwords
df_vader_In['review'] = df_vader_In['review'].apply(lambda x: ' '.join([word for word in x.split() if word not in stop_words]))

# Ver las primeras filas para verificar el resultado
print(df_vader_In.head())

In [ ]:
# Analizar el sentimiento
#sentiment = sia.polarity_scores(df_vader_In['answer'])
df_vader_In['sentimiento'] = sia.polarity_scores(df_vader_In['answer'])
df_vader_In['sentimiento'] = df_vader_In['sentimiento'].apply(lambda x: x['compound'])


# Imprimir los resultados
print("Puntuaciones de sentimiento:", sentiment)


In [ ]:
# Aplicar el análisis de sentimiento
df['sentimiento'] = df['comentario'].apply(analizar_sentimiento)

# Extraer la puntuación compuesta
df['compuesto'] = df['sentimiento'].apply(lambda x: x['compound'])

# Clasificar el sentimiento
df['interpretacion'] = df['compuesto'].apply(lambda x: 'Positivo' if x > 0.05 else ('Negativo' if x < -0.05 else 'Neutro'))

# Mostrar resultados
print(df[['comentario', 'sentimiento', 'compuesto', 'interpretacion']])


In [ ]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt